# Test for Fixed Effects

## Load Libraries and Dataset

### Libraries

In [30]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from linearmodels.panel import PooledOLS

### Dataset and PanelOLS Object

In [31]:
sgx = pd.read_csv("data/clean_sgx.csv", index_col= None)
sgx = sgx[['Company Code', 'Year', 'LEVERAGE',
           'SIZE', 'PROFITABILITY', 'TANG', 'LIQUID', 'MCAP', 'SOLV']]
sgx['year'] = pd.Categorical(sgx['Year'])
sgx['compcode'] = pd.Categorical(sgx['Company Code'])
sgx = sgx.set_index(['Company Code', 'Year'])

endo = sgx.LEVERAGE
exog_vars = ['SIZE', 'PROFITABILITY', 'TANG', 'LIQUID', 'MCAP', 'SOLV']
exog = sm.add_constant(sgx[exog_vars])

pooled_mod = PooledOLS(endo, exog)
pooled_res = pooled_mod.fit()

## F-test for Fixed Effects

In [28]:
def fixedeffect(endo = endo, exog = exog,
                entity_eff = True, time_eff = True):
    mod = PanelOLS(endo, exog,
                   entity_effects=entity_eff,
                   time_effects=time_eff)
    res = mod.fit()

    return res

### Joint test for two-way FE

In [32]:
fixedeffect().summary

Dep. Variable:,LEVERAGE,R-squared:,0.2294
Estimator:,PanelOLS,R-squared (Between):,0.1387
No. Observations:,1728,R-squared (Within):,0.2397
Date:,"Sun, Mar 31 2024",R-squared (Overall):,0.1569
Time:,17:15:03,Log-likelihood,1628.0
Cov. Estimator:,Unadjusted,,
,,F-statistic:,74.386
Entities:,216,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(6,1499)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,74.386


Joint F-statistic = F-statistic (robust) = **74.386** with p-value smaller than 0.0001 (99.99% confidence level).

Highly significant that the two-way fixed effect exists compared to the PooledOLS null hypothesis.

### Conditional test for entity FE

In [22]:
fixedeffect(entity_eff=True, time_eff=False).summary

Dep. Variable:,LEVERAGE,R-squared:,0.2401
Estimator:,PanelOLS,R-squared (Between):,0.1308
No. Observations:,1728,R-squared (Within):,0.2401
Date:,"Sun, Mar 31 2024",R-squared (Overall):,0.1504
Time:,16:33:13,Log-likelihood,1615.1
Cov. Estimator:,Unadjusted,,
,,F-statistic:,79.286
Entities:,216,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(6,1506)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,79.286


F-statistic = **79.286** for conditional entity effect.

### Conditional test for time FE

In [23]:
fixedeffect(entity_eff=False, time_eff=True).summary

Dep. Variable:,LEVERAGE,R-squared:,0.3616
Estimator:,PanelOLS,R-squared (Between):,0.4295
No. Observations:,1728,R-squared (Within):,0.0537
Date:,"Sun, Mar 31 2024",R-squared (Overall):,0.3621
Time:,16:37:08,Log-likelihood,285.77
Cov. Estimator:,Unadjusted,,
,,F-statistic:,161.80
Entities:,216,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(6,1714)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,161.80


F-statistic = **161.80** for conditional time effect.

### Marginal test for entity FE

In [26]:
exog_vars_marginal_entity = ['SIZE', 'PROFITABILITY', 'TANG', 'LIQUID', 'MCAP', 'SOLV', 'year']
exog_marginal_entity = sm.add_constant(sgx[exog_vars_marginal_entity])
fixedeffect(exog = exog_marginal_entity, time_eff = False).summary

Dep. Variable:,LEVERAGE,R-squared:,0.2513
Estimator:,PanelOLS,R-squared (Between):,0.1387
No. Observations:,1728,R-squared (Within):,0.2513
Date:,"Sun, Mar 31 2024",R-squared (Overall):,0.1589
Time:,16:41:44,Log-likelihood,1628.0
Cov. Estimator:,Unadjusted,,
,,F-statistic:,38.706
Entities:,216,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(13,1499)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,38.706


F-statistic = **38.706** for marginal entity effect.

### Marginal test for time FE

In [27]:
exog_vars_marginal_time = ['SIZE', 'PROFITABILITY', 'TANG', 'LIQUID', 'MCAP', 'SOLV', 'compcode']
exog_marginal_time = sm.add_constant(sgx[exog_vars_marginal_time])
fixedeffect(exog = exog_marginal_time, entity_eff = False).summary

Dep. Variable:,LEVERAGE,R-squared:,0.8650
Estimator:,PanelOLS,R-squared (Between):,1.0000
No. Observations:,1728,R-squared (Within):,0.2397
Date:,"Sun, Mar 31 2024",R-squared (Overall):,0.8636
Time:,16:43:40,Log-likelihood,1628.0
Cov. Estimator:,Unadjusted,,
,,F-statistic:,43.449
Entities:,216,P-value,0.0000
Avg Obs:,8.0000,Distribution:,"F(221,1499)"
Min Obs:,8.0000,,
Max Obs:,8.0000,F-statistic (robust):,43.449


F-statistic = **43.449** for marginal entity effect.

## LR test for Restricted and Unrestricted models

In [33]:
# Joint LR-test
unres_loglikehood = fixedeffect().loglik
res_loglikehood = pooled_res.loglik

lr_ratio = -2*(res_loglikehood - unres_loglikehood)
lr_df = pooled_res.df_model
lr_pval = 1 - stats.chi2.cdf(lr_ratio, lr_df)

print('LR test for 2FE')
print(f'Log-likelihood Ratio: {round(lr_ratio, 4)}')
print(f'p-value: {round(lr_pval, 4)}')
print(f'Distribution: chisq({lr_df})')

LR test for 2FE
Log-likelihood Ratio: 2691.9046
p-value: 0.0
Distribution: chisq(7)
